In [1]:
import ncar_jobqueue

In [2]:
cluster = ncar_jobqueue.NCARCluster(
#    cores=9,  # The number of cores you want
    memory='60GB',  # Amount of memory
#    processes=9,  # How many processes
    walltime='06:00:00',  # Amount of wall time
    project="ncgd0011",  #Project id
)

cluster.scale(16)

/glade/u/home/pmora/.conda/envs/etpac/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37735 instead
  warnings.warn(


In [3]:
%matplotlib inline
%load_ext autoreload
%load_ext watermark


import dask
import distributed
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import xarray as xr
#import xgcm
import cartopy.crs as cr
from cartopy import feature as cf
import nctoolkit as nct

mpl.rcParams["savefig.dpi"] = 300
mpl.rcParams["savefig.bbox"] = "tight"
mpl.rcParams["figure.dpi"] = 300

xr.set_options(keep_attrs=True, display_style="html")

%watermark -iv

Please install CDO version 1.9.7 or above: https://code.mpimet.mpg.de/projects/cdo/ or https://anaconda.org/conda-forge/cdo
json         : 2.0.9
ncar_jobqueue: 2021.4.14
nctoolkit    : 0.6.3
sys          : 3.9.4 | packaged by conda-forge | (default, May 10 2021, 22:13:33) 
[GCC 9.3.0]
matplotlib   : 3.5.3
pandas       : 1.4.4
dask         : 2021.6.0
distributed  : 2021.6.0
xarray       : 0.18.2
cartopy      : 0.19.0.post1
numpy        : 1.20.3



In [4]:
client = distributed.Client(cluster)
client

Client Scheduler: tcp://10.12.206.54:37152 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/pmora/proxy/37735/status,Cluster Workers: 16 Cores: 32 Memory: 400.00 GiB


In [5]:
files1 = '/glade/campaign/cgd/oce/people/bachman/ETP_1_20_tides/HOURLY/ocean_hourly__*.nc'
files2 = '/glade/campaign/cgd/oce/people/bachman/ETP_1_20_tides/DAILY/ocean_daily__*.nc'

In [6]:
DS1= xr.open_mfdataset(
    files1,
    compat="override",
    data_vars="minimal",
    coords="minimal",
    parallel=True,
    concat_dim="time",
    combine="nested",
    chunks={  "yh":200, "xh":200, },
)[["wind_stress_curl"]]
#[["so", "thetao", "uo", "vo", "wo"]]

In [18]:
DS2 = xr.open_mfdataset(
    files2,
    compat="override",
    data_vars="minimal",
    coords="minimal",
    parallel=True,
    concat_dim="time",
    combine="nested",
    chunks={"xh": 200, "yh": 400, "z_i": 400},
)[["wo"]]
#[["so", "thetao", "uo", "vo", "wo"]]

In [7]:
DS1

,Array,Chunk
Bytes,764.11 GiB,3.66 MiB
Shape,"(227904, 900, 1000)","(24, 200, 200)"
Count,484296 Tasks,237400 Chunks
Type,float32,numpy.ndarray


In [ ]:
DS2



¡¡¡¡¡¡¡¡WIND STRESS CURL!!!!




In [8]:
tau = DS1['wind_stress_curl']
tau

,Array,Chunk
Bytes,764.11 GiB,3.66 MiB
Shape,"(227904, 900, 1000)","(24, 200, 200)"
Count,484296 Tasks,237400 Chunks
Type,float32,numpy.ndarray


In [14]:
A2_tau = tau.sel(xh=np.arange(260,283), yh=np.arange(0,17), method='nearest')
A2_tau

/glade/u/home/pmora/.local/lib/python3.9/site-packages/xarray/core/indexing.py:1385: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]


<xarray.DataArray 'wind_stress_curl' (time: 227904, yh: 17, xh: 23)>
dask.array<getitem, shape=(227904, 17, 23), dtype=float32, chunksize=(24, 10, 10), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 260.0 261.0 262.0 263.0 ... 279.0 280.0 281.0 282.0
  * yh       (yh) float64 0.025 1.025 2.025 3.025 ... 13.02 14.02 15.02 16.02
  * time     (time) datetime64[ns] 1992-12-30T01:00:00 ... 2018-12-30
Attributes:
    long_name:  Wind stress curl
    units:      Pa m-1

In [15]:
A2_tau_day = A2_tau.groupby('time.date').mean('time').persist()

In [16]:
A2_tau_day

<xarray.DataArray 'wind_stress_curl' (date: 9497, yh: 17, xh: 23)>
dask.array<stack, shape=(9497, 17, 23), dtype=float32, chunksize=(1, 10, 10), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 260.0 261.0 262.0 263.0 ... 279.0 280.0 281.0 282.0
  * yh       (yh) float64 0.025 1.025 2.025 3.025 ... 13.02 14.02 15.02 16.02
  * date     (date) object 1992-12-30 1992-12-31 ... 2018-12-29 2018-12-30
Attributes:
    long_name:  Wind stress curl
    units:      Pa m-1

In [17]:
A2_tau_day['date'] = pd.DatetimeIndex(A2_tau_day['date'].values)

In [18]:
A2_tau_day

<xarray.DataArray 'wind_stress_curl' (date: 9497, yh: 17, xh: 23)>
dask.array<stack, shape=(9497, 17, 23), dtype=float32, chunksize=(1, 10, 10), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 260.0 261.0 262.0 263.0 ... 279.0 280.0 281.0 282.0
  * yh       (yh) float64 0.025 1.025 2.025 3.025 ... 13.02 14.02 15.02 16.02
  * date     (date) datetime64[ns] 1992-12-30 1992-12-31 ... 2018-12-30
Attributes:
    long_name:  Wind stress curl
    units:      Pa m-1

In [ ]:
day_wind_stress_curl = A2_tau_day.to_netcdf('/glade/scratch/pmora/day_wind_stress_curl.nc')





¡¡¡¡¡¡VERTICAL VELOCITY!!!!!!!!!!






In [33]:
A2 = DS2.isel(z_i=slice(9))
A2

,Array,Chunk
Bytes,286.54 GiB,2.75 MiB
Shape,"(9496, 9, 900, 1000)","(1, 9, 400, 200)"
Count,436816 Tasks,142440 Chunks
Type,float32,numpy.ndarray


In [34]:
A2 = A2.sel(xh=np.arange(260,283,0.05), yh=np.arange(0,17,0.05), method='nearest')

/glade/u/home/pmora/.local/lib/python3.9/site-packages/xarray/core/indexing.py:1385: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]


In [35]:
A2_wo = A2["wo"]
A2_wo

,Array,Chunk
Bytes,49.79 GiB,1.65 MiB
Shape,"(9496, 9, 340, 460)","(1, 9, 240, 200)"
Count,579256 Tasks,56976 Chunks
Type,float32,numpy.ndarray


In [43]:
A2_wo_day = A2_wo.groupby('time.date').mean('z_i')*86400
A2_wo_day

<xarray.DataArray 'wo' (time: 9496, yh: 340, xh: 460)>
dask.array<mul, shape=(9496, 340, 460), dtype=float64, chunksize=(1, 240, 200), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 260.0 260.1 260.1 260.2 ... 282.8 282.9 282.9 283.0
  * yh       (yh) float64 0.025 0.075 0.125 0.125 ... 16.82 16.88 16.93 16.93
  * time     (time) datetime64[ns] 1992-12-30T12:00:00 ... 2018-12-29T12:00:00
Attributes:
    long_name:  Sea water Z velocity
    units:      m s-1

In [40]:
A2_wo_day[0]

<xarray.DataArray 'wo' (yh: 340, xh: 460)>
dask.array<getitem, shape=(340, 460), dtype=float64, chunksize=(240, 200), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 260.0 260.1 260.1 260.2 ... 282.8 282.9 282.9 283.0
  * yh       (yh) float64 0.025 0.075 0.125 0.125 ... 16.82 16.88 16.93 16.93
    time     datetime64[ns] 1992-12-30T12:00:00
Attributes:
    long_name:  Sea water Z velocity
    units:      m s-1

In [41]:
A2_tau_day[-1]

<xarray.DataArray 'wind_stress_curl' (yh: 340, xh: 460)>
dask.array<getitem, shape=(340, 460), dtype=float32, chunksize=(200, 200), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 260.0 260.1 260.1 260.2 ... 282.8 282.9 282.9 283.0
  * yh       (yh) float64 0.025 0.075 0.125 0.125 ... 16.82 16.88 16.93 16.93
    date     object 2018-12-30
Attributes:
    long_name:  Wind stress curl
    units:      Pa m-1